In [8]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import pandas as pd
from skimage.transform import resize
import nrrd
import os
import glob
import mahotas as mh

In [4]:
DATAPATH = '/raid/mpsych/CACTAS/DATA/'
ESUSPATH = '/raid/mpsych/CACTAS/DATA/ESUS/'
IMGPATH = '/raid/mpsych/CACTAS/DATA/ESUS_NPY/IMG'
LBPATH = '/raid/mpsych/CACTAS/DATA/ESUS_NPY/LABEL'

In [ ]:
# Convert nrrd to npy and save file

In [3]:
for f in os.listdir(ESUSPATH):
    if f.endswith('img.nrrd'):
        current_id = f.split('.')[0]
        im, h = nrrd.read(ESUSPATH + f'{current_id}.img.nrrd')
        imarray = np.array(im)
        
        #if f'{current_id}.npy' not in DATAPATH + 'ESUS_NPY/IMG':
        #    np.save(DATAPATH + 'ESUS_NPY/IMG' + f'/{current_id}.npy', imarray)
    if f.endswith('seg.nrrd'):
        current_id = f.split('.')[0]
        im, h = nrrd.read(ESUSPATH + f'{current_id}.img.nrrd')
        imarray = np.array(im)

        #if f'{current_id}.npy' not in DATAPATH + 'ESUS_NPY/LABEL':
        #    np.save(DATAPATH + 'ESUS_NPY/LABEL' + f'/{current_id}.npy', imarray)

In [2]:
'''
for i in range(5):
    for f in os.listdir(IMGPATH):
        current_id = f.split('.')[0]
        img = np.load(IMGPATH + f'/{current_id}.npy', allow_pickle=True)

        print(img.shape)
        print(img)
'''

"\nfor i in range(5):\n    for f in os.listdir(IMGPATH):\n        current_id = f.split('.')[0]\n        img = np.load(IMGPATH + f'/{current_id}.npy', allow_pickle=True)\n\n        print(img.shape)\n        print(img)\n"

In [ ]:
# make array list

In [29]:
loadedimages = []

for f in os.listdir(IMGPATH):
    current_id = f.split('.')[0]
    loadedimages.append(np.load(IMGPATH + f'/{current_id}.npy', allow_pickle=True).copy())

#imgs = np.asarray(loadedimages)

In [ ]:
loadedmasks = []

for f in os.listdir(LBPATH):
    current_id = f.split('.')[0]
    loadedmasks += [np.load(LBPATH + f'/{current_id}.npy', allow_pickle=True).copy()]
    
masks = np.asarray(loadedmasks)

In [ ]:
masks

In [65]:
DATAPATH = '/raid/mpsych/CACTAS/DATA/ESUS/'

In [66]:
DATASETS = sorted(v for v in os.listdir(DATAPATH))

In [77]:
len_d = len(DATASETS)

In [78]:
len_d // 2

66

In [12]:
images = []
labels = []

image_patches = []
label_patches = []

DATAPATH = '/raid/mpsych/CACTAS/DATA/ESUS/'
DATASETS = sorted(v for v in os.listdir(DATAPATH))
for i,d in enumerate(DATASETS):
    #print(i, d) # 0 - 131
    D = DATASETS[i]

    files = glob.glob(DATAPATH + "*.nrrd")
    label_file = [os.path.join(DATAPATH, D, s) for s in files if 'seg' in s]
    image_file = [os.path.join(DATAPATH, D, i) for i in files if not 'seg' in i]

    label_file = [s for s in label_file if os.path.isfile(s)][i]
    image_file = [i for i in image_file if os.path.isfile(i)][i]

    #print(i)
    #print(image_file)
    #print(label_file)

    label_with_header = nrrd.read(label_file)
    image_with_header = nrrd.read(image_file)

    label = label_with_header[0]
    image = image_with_header[0]
    #print("img, label: ", image, label)

    # normalize grayscale to 0..1
    image_normalized = image.copy().astype(np.float64)
    image_normalized = (image_normalized - image_normalized.min()) / (image_normalized.max() - image_normalized.min())
    #print(image_normalized)

    # remove empty slices
    filtered_image = image_normalized.copy()
    filtered_label = label.copy()

    slices_to_remove = []
    
    #print(filtered_label.shape)
    '''
    (512, 512, 258)
    (512, 512, 255)
    (512, 512, 216)
    (512, 512, 245)
    (512, 512, 237)
    (512, 512, 221)
    (512, 512, 390)
    (512, 512, 268)
    (512, 512, 216)
    (512, 512, 221)
    (512, 512, 240)
    (512, 512, 201)
    (512, 512, 217)
    (512, 512, 237)
    (512, 512, 211)
    (512, 512, 236)
    (512, 512, 245)
    (512, 512, 293)
    (512, 512, 264)
    (512, 512, 176)
    (512, 512, 261)
    (512, 512, 176)
    (512, 512, 179)
    (512, 512, 232)
    '''

    for z in range(filtered_label.shape[2]):
        if filtered_label[:,:,z].max() == 0:
            slices_to_remove.append(z)
    
    #print(slices_to_remove)

    slices_to_remove = sorted(slices_to_remove, reverse=True)
    
    '''
    for z in slices_to_remove:
        filtered_image = np.delete(filtered_image, z, 2)
        filtered_label = np.delete(filtered_label, z, 2)
    '''
    
    for z in slices_to_remove:
        if z < filtered_image.shape[2]:
            filtered_image = np.delete(filtered_image, z, 2)
            filtered_label = np.delete(filtered_label, z, 2)

    #print("img: ", filtered_image)
    #print("lb: ", filtered_label)


    # loop thru each slice
    for z in range(filtered_label.shape[2]):

        image_slice = filtered_image[:,:,z:z+1]
        label_slice = filtered_label[:,:,z:z+1]

        # relabel the segments
        relabel, obj = mh.label(label_slice)

        #print('relabeled')

        # loop thru each segment, binarize and crop
        for i in range(1, relabel.max()+1):

            single_label = np.zeros((relabel.shape), relabel.dtype)
            single_label[relabel == i] = 1

            #image_cropped, single_label_cropped = C.Util.crop(image_slice, single_label, target_size=32)
            #def crop(image, label, increase_xy=10, increase_z=0, target_size=None, just_z=False):
            
            increase_xy=10
            increase_z=0
            target_size=32
            just_z=False
            
            bbox = mh.bbox(single_label) # ignore large portion of label since its all 0
                          # to only include annotated plaque regions

            increase_x = increase_xy
            increase_y = increase_xy

            if target_size:
                widthY = bbox[1]-bbox[0]
                widthX = bbox[3]-bbox[2]

                increase_x = (target_size - widthX) // 2
                increase_y = (target_size - widthY) // 2

                bufferY = 0
                if (bbox[0]-increase_y + bbox[1]+increase_y) != target_size:
                    bufferY = 1

                bufferX = 0
                if (bbox[2]-increase_x + bbox[3]+increase_x) != target_size:
                    bufferX = 1
                
            if just_z:
                increase_xy = 0
                bbox[0] = 0
                bbox[1] = image_slice.shape[0]
                bbox[2] = 0
                bbox[3] = image_slice.shape[1]


                # crop label and image according to bbox but make it a little larger
                label_cropped = single_label[bbox[0]-increase_y:bbox[1]+increase_y+bufferY, 
                                  bbox[2]-increase_x:bbox[3]+increase_x+bufferX,
                                  bbox[4]-increase_z:bbox[5]+increase_z]
                image_cropped = image_slice[bbox[0]-increase_y:bbox[1]+increase_y+bufferY, 
                                  bbox[2]-increase_x:bbox[3]+increase_x+bufferX,
                                  bbox[4]-increase_z:bbox[5]+increase_z]




            image_patches.append(image_cropped)
            label_patches.append(single_label_cropped)


NameError: name 'image_cropped' is not defined